# A seminar focused on data collection and analysis practices to foster researchers' AI application capabilities.

### "This is a notebook to help you practice. Just follow along, one cell at a time. Once you're done, come on back here, and there'll be lots more for you to discover!"

## 1. 제조업 디지털 전환과 AI의 필요성: 왜 제조업에 AI가 필요한가?

- 생산 효율성 향상, 품질 관리, 설비 예측 보전 등 AI의 핵심 적용 사례
- 정형 데이터 (센서, PLC, MES 등) 및 비정형 데이터 (이미지, 영상, 음성 등)의 이해.
- 실제 제조 현장 데이터 사례 분석 및 데이터 특성 파악.

### a. 제조 데이터 수집 및 전처리:

- Arduino
- ESP32

### b. Blink로 시작하기

- 먼저 Visual Studio Code (https://code.visualstudio.com/) 를 설치한다.
- Extensions에서 "PlatformIO IDE"를 검색해서 설치한다.

    <img src="images/image_000.png" alt="Extensions에서 PlatformIO IDE를 검색해서 설치한다.">

<br>

- 프로젝트를 생성한 후 아래의 코드를 입력하고 실행시켜 본다.

In [ ]:
#include "Arduino.h"

#define led_builtin 2

void setup() {
  pinMode(led_builtin, OUTPUT);
}

void loop() {
  digitalWrite(led_builtin, HIGH);
  delay(1000);
  digitalWrite(led_builtin, LOW);
  delay(1000);
}

- 가변저항기를 연결해본다.
- 가변저항기의 저항을 변경시켜 깜빡이는 주기를 바꿔본다.

In [ ]:
#include "Arduino.h"

#define led_builtin 2
#define AdIN 33

void setup() {
  pinMode(led_builtin, OUTPUT);
  pinMode(AdIN, INPUT);
}

void loop() {
  int d_time = analogRead(AdIN) / 4;
  digitalWrite(led_builtin, HIGH);
  delay(d_time);
  digitalWrite(led_builtin, LOW);
  delay(d_time);
}

### c. DHT 센서를 연결해보자.

- 묻지마 3단계
- 일단 먼저 실행하고 코드는 실행결과를 본 다음에 보자.
- 커넥터의 핀 번호는 아래의 그림을 참조.

    <img src="images/image_001.png">
<br>

- 센서 데이터 선은 32번에 연결하고, 아래의 코드를 실행시켜 본다.

In [ ]:
#include "DHT.h"

#define DHTPIN 32
#define DHTTYPE DHT11

DHT dht(DHTPIN, DHTTYPE);

void setup() {
  Serial.begin(9600);
  Serial.println(F("DHTxx test!"));

  dht.begin();
}

void loop() {
  delay(2000);

  float h = dht.readHumidity();
  float t = dht.readTemperature();
  float f = dht.readTemperature(true);

  if (isnan(h) || isnan(t) || isnan(f)) {
    Serial.println(F("Failed to read from DHT sensor!"));
    return;
  }

  float hif = dht.computeHeatIndex(f, h);
  float hic = dht.computeHeatIndex(t, h, false);

  Serial.print(F("Humidity: "));
  Serial.print(h);
  Serial.print(F("%  Temperature: "));
  Serial.print(t);
  Serial.print(F("°C "));
  Serial.print(f);
  Serial.print(F("°F  Heat index: "));
  Serial.print(hic);
  Serial.print(F("°C "));
  Serial.print(hif);
  Serial.println(F("°F"));
}

### d. MPU6050 실습

- MPU6050은 가속도계와 자이로스코프가 결합된 6축(X, Y, Z) 모션 트래킹 센서이다. 이 센서는 이 두 가지 주요 기능을 한 칩에 통합하여, 물체의 움직임과 방향을 정밀하게 측정할 수 있다. 🚀

- 주요 기능
    * 3축 자이로스코프 (3-Axis Gyroscope): 물체의 회전 속도를 측정한다. 🕹️ 예를 들어, 드론이 어느 방향으로 얼마나 빠르게 회전하고 있는지를 파악하는 데 사용할 수 있다.

    * 3축 가속도계 (3-Axis Accelerometer): 물체에 작용하는 가속도를 측정한다. 🏃‍♂️ 스마트폰이 기울어지는 각도를 감지하거나, 로봇의 움직임을 제어하는 데 사용할 수 있고, 가속도의 변화를 빠르게 측정하면 기계의 진동을 측정할 수도 있다.

- 특징 및 활용 분야
    * I2C 통신: MPU6050은 I2C(Inter-Integrated Circuit) 통신 방식을 사용하여 마이크로컨트롤러(예: 아두이노, 라즈베리 파이)와 쉽게 연결할 수 있다.

    * 소형화 및 저전력: 작은 크기와 낮은 전력 소비량.. 따라서 다양한 휴대용 기기에 적합하다.

- 활용 분야: 드론, 스마트폰, 웨어러블 기기, 로봇 공학 등 물체의 자세와 움직임을 감지해야 하는 다양한 분야에서 널리 사용된다.

- 이 센서의 핵심은 하나의 칩에 두 가지 중요한 센서를 통합하여, 물체의 "방향(자이로스코프)"과 "움직임(가속도계)"을 동시에 파악할 수 있다는 점이다.

    <img src="https://lastminuteengineers.com/wp-content/uploads/arduino/MPU6050-Module-Gyroscope-Axis.jpg">

- 센서를 ESP32에 연결한다.

    <img src="images/image_003.png">

<br>

- 연결되었으면 아래의 코드를 실행한다.

In [ ]:
#include "Arduino.h"
#include "Adafruit_MPU6050.h"
#include "Adafruit_Sensor.h"
#include "Wire.h"

Adafruit_MPU6050 mpu;

void setup(void) {
  Serial.begin(115200);
  while (!Serial)
    delay(10); // will pause Zero, Leonardo, etc until serial console opens

  Serial.println("Adafruit MPU6050 test!");

  // Try to initialize!
  if (!mpu.begin()) {
    Serial.println("Failed to find MPU6050 chip");
    while (1) {
      delay(10);
    }
  }
  Serial.println("MPU6050 Found!");

  mpu.setAccelerometerRange(MPU6050_RANGE_8_G);
  Serial.print("Accelerometer range set to: ");
  switch (mpu.getAccelerometerRange()) {
  case MPU6050_RANGE_2_G:
    Serial.println("+-2G");
    break;
  case MPU6050_RANGE_4_G:
    Serial.println("+-4G");
    break;
  case MPU6050_RANGE_8_G:
    Serial.println("+-8G");
    break;
  case MPU6050_RANGE_16_G:
    Serial.println("+-16G");
    break;
  }
  mpu.setGyroRange(MPU6050_RANGE_500_DEG);
  Serial.print("Gyro range set to: ");
  switch (mpu.getGyroRange()) {
  case MPU6050_RANGE_250_DEG:
    Serial.println("+- 250 deg/s");
    break;
  case MPU6050_RANGE_500_DEG:
    Serial.println("+- 500 deg/s");
    break;
  case MPU6050_RANGE_1000_DEG:
    Serial.println("+- 1000 deg/s");
    break;
  case MPU6050_RANGE_2000_DEG:
    Serial.println("+- 2000 deg/s");
    break;
  }

  mpu.setFilterBandwidth(MPU6050_BAND_21_HZ);
  Serial.print("Filter bandwidth set to: ");
  switch (mpu.getFilterBandwidth()) {
  case MPU6050_BAND_260_HZ:
    Serial.println("260 Hz");
    break;
  case MPU6050_BAND_184_HZ:
    Serial.println("184 Hz");
    break;
  case MPU6050_BAND_94_HZ:
    Serial.println("94 Hz");
    break;
  case MPU6050_BAND_44_HZ:
    Serial.println("44 Hz");
    break;
  case MPU6050_BAND_21_HZ:
    Serial.println("21 Hz");
    break;
  case MPU6050_BAND_10_HZ:
    Serial.println("10 Hz");
    break;
  case MPU6050_BAND_5_HZ:
    Serial.println("5 Hz");
    break;
  }

  Serial.println("");
  delay(100);
}

void loop() {

  /* Get new sensor events with the readings */
  sensors_event_t a, g, temp;
  mpu.getEvent(&a, &g, &temp);

  /* Print out the values */
  Serial.print("Acceleration X: ");
  Serial.print(a.acceleration.x);
  Serial.print(", Y: ");
  Serial.print(a.acceleration.y);
  Serial.print(", Z: ");
  Serial.print(a.acceleration.z);
  Serial.println(" m/s^2");

  Serial.print("Rotation X: ");
  Serial.print(g.gyro.x);
  Serial.print(", Y: ");
  Serial.print(g.gyro.y);
  Serial.print(", Z: ");
  Serial.print(g.gyro.z);
  Serial.println(" rad/s");

  Serial.print("Temperature: ");
  Serial.print(temp.temperature);
  Serial.println(" degC");

  Serial.println("");
  delay(1);
}

### e. WIFI를 통한 데이터 전송 실습 (Thinger.io를 이용)

- ESP32는 wifi 통신 기능을 가지고 있으므로 Wifi를 이용해서 인터넷에 접속할 수 있다.
- 인터넷에 접속한 후 IoT 서버에 데이트를 전송하는 것이 가능하다.
- 인터넷 상에는 IoT 플렛폼이 많이 있다. 그중에 특히 Open source (다른 말로 공짜!!) 플랫폼들도 제법 많이 있다.
- 그중에 우리는 Thinger.io 플랫폼을 이용해보자.
- 먼저 Thinger.io(https://thinger.io/)에 접속하여 회원가입을 하고 로그인 해보자.

    <img src="images/image_004.png" width="600"><br>
- Device를 생성한다.
- ESP32 코드를 작성한다. (코드는 아래 코드를 이용)
    * 묻지마 3단계의 의해서 라이브러리를 설치하고 example code를 가져와서 실행한다.

In [ ]:
//================================
#define THINGER_SERIAL_DEBUG
#include "ThingerESP32.h"

#define USERNAME "greathtj"
#define DEVICE_ID "ESP32_htj_001"
#define DEVICE_CREDENTIAL "1234567890"

#define SSID "GreatRedme"
#define SSID_PASSWORD "kt9354qq"

ThingerESP32 thing(USERNAME, DEVICE_ID, DEVICE_CREDENTIAL);

//================================
#include "DHT.h"
#define DHTPIN 32
#define DHTTYPE DHT11

DHT dht(DHTPIN, DHTTYPE);

float t, h;

//================================
void setup() {
  Serial.begin(115200);
  dht.begin();

  thing.add_wifi(SSID, SSID_PASSWORD);

  // resource output example (i.e. reading a sensor value)
  thing["temperature"] >> outputValue(t);
  thing["humidity"] >> outputValue(h);
}

//================================
void loop() {
  thing.handle();
  delay(1000);

  h = dht.readHumidity();
  t = dht.readTemperature();

  if (isnan(h) || isnan(t)) {
    Serial.println(F("Failed to read from DHT sensor!"));
    return;
  }

  float hic = dht.computeHeatIndex(t, h, false);

  Serial.print(F("Humidity: "));
  Serial.print(h);
  Serial.print(F("%  Temperature: "));
  Serial.print(t);
  Serial.print(F("°C "));
  Serial.println();
}

- 정상적으로 작동하고 있으면 Thinger.io에서 Devices 메뉴를 눌러 확인해보면 아래와 같이 나의 디바이스가 연결된 것을 확인할 수 있다.

    <img src="images/image_005.png" width="600"><br>

- 그리고 Statics 메뉴를 클릭하면 데이터가 발생하고 있는 위치를 세계지도에 표시해 준다.

    <img src="images/image_006.png" width="600"><br>

- 이제 Data Bucket (즉, 데이터베이스 테이블이다.)을 만들어주자. (안타깝지만 1분에 한개의 데이터만 받을 수 있다. 공짜 아닌가 공짜.)

    <img src="images/image_007.png" width="600"><br>

- 이렇게 데이터가 쌓이기 시작했다. (달랑 두 포인트이기는 하지만... 시작이 반이다.)

    <img src="images/image_008.png" width="600"><br>

- 동일한 방법으로 "humidity"도 수집하면 된다.
- 데이터가 수집되고 있으니 그래프로도 볼 수 있지 않을까?
    - Dashboards 메뉴를 선택한다.
    - Dashboard를 하나 생성한다.
    - 생성된 Dashboard에 위젯추가하기("+ Add Widget")를 클릭한다.

        <img src="images/image_009.png" width="600"> <br>

    - 그리고 Time Series Chart를 선택해서 시계열 온도데이터 혹은 습도데이터 그래프를 만들자.

        <img src="images/image_010.png" width="600"> <br>

    - 나름대로는 professional한 모니터링 시스템이 되었다. (그럼 다 배웠으니, 이만 하산하도록....)

        <img src="images/image_012.png" width="600">


### f. 내 서버를 가질 수는 없을까?
- 많은 기업들은 데이터가 어딘가 외부로 나가는 것을 꺼려한다.
- 클라우드에 저장하는 것이 안전한가, 자체적으로 가지고 있는 것이 안전한가에 대한 논쟁은 뒤로 하고, 아무튼 내부에 두는 것도 할 수는 있어야 하지 않겠는가.
- MQTT, Node-RED, 그리고 자체적인 DB (예컨대, InfluxDB)를 이용할 수 있다. (아래 그림 참조)

    <img src="https://i0.wp.com/randomnerdtutorials.com/wp-content/uploads/2018/05/ESP32-MQTT-bme280.png?w=800&quality=100&strip=all&ssl=1">

- 미리 마련된 서버(생기원 혹은 중소기업현장?)에 MQTT와 Node-RED, 그리고 InfluxDB를 설치해 두고, 그걸 이용해 보자.
    - 코드는 아래의 코드를 실행해서 테스트 한다.
    - 5초마다 한번씩 데이터를 보낸다.. 아니 1초마다 한번씩도 보낼 수도 있다. 다만 인터넷 속도가 문제일 뿐.

In [ ]:
#include "WiFi.h"
#include "PubSubClient.h"
#include "Wire.h"

const char* ssid = "GreatRedme";
const char* password = "kt9354qq";
const char* mqtt_server = "kiotech.gonetis.com";
// const char* my_mqtt_id = "esp32_htj";
// const char* my_subscribe = "esp32_htj/output";
// const char* publish_t = "esp32_htj/temperature";
// const char* publish_h = "esp32_htj/humidity";

WiFiClient espClient;
PubSubClient client(espClient);
long lastMsg = 0;
char msg[50];
int value = 0;

#include "DHT.h"
#define DHTPIN 32
#define DHTTYPE DHT11

DHT dht(DHTPIN, DHTTYPE);

float t, h;

void setup_wifi() {
  delay(10);
  // We start by connecting to a WiFi network
  Serial.println();
  Serial.print("Connecting to ");
  Serial.println(ssid);

  WiFi.begin(ssid, password);

  while (WiFi.status() != WL_CONNECTED) {
    delay(500);
    Serial.print(".");
  }

  Serial.println("");
  Serial.println("WiFi connected");
  Serial.println("IP address: ");
  Serial.println(WiFi.localIP());
}

void callback(char* topic, byte* message, unsigned int length) {
  Serial.print("Message arrived on topic: ");
  Serial.print(topic);
  Serial.print(". Message: ");
  String messageTemp;
  
  for (int i = 0; i < length; i++) {
    Serial.print((char)message[i]);
    messageTemp += (char)message[i];
  }
  Serial.println();

}

void reconnect() {
  // Loop until we're reconnected
  while (!client.connected()) {
    Serial.print("Attempting MQTT connection...");
    // Attempt to connect
    if (client.connect("esp32_htj")) {
      Serial.println("connected");
      // Subscribe
      client.subscribe("esp32_htj/output");
    } else {
      Serial.print("failed, rc=");
      Serial.print(client.state());
      Serial.println(" try again in 5 seconds");
      // Wait 5 seconds before retrying
      delay(5000);
    }
  }
}

void setup() {
  Serial.begin(115200);
  dht.begin();

  setup_wifi();
  client.setServer(mqtt_server, 1883);
  client.setCallback(callback);
}

void loop() {
  if (!client.connected()) {
    reconnect();
  }
  client.loop();
  delay(1000);

  h = dht.readHumidity();
  t = dht.readTemperature();

  if (isnan(h) || isnan(t)) {
    Serial.println(F("Failed to read from DHT sensor!"));
    return;
  }

  float hic = dht.computeHeatIndex(t, h, false);

  Serial.print(F("Humidity: "));
  Serial.print(h);
  Serial.print(F("%  Temperature: "));
  Serial.print(t);
  Serial.print(F("°C "));
  Serial.println();

  long now = millis();
  if (now - lastMsg > 5000) {
    lastMsg = now;
    
    char tempString[8];
    dtostrf(t, 1, 2, tempString);
    client.publish("esp32_htj/temperature", tempString);

    char humString[8];
    dtostrf(h, 1, 2, humString);
    client.publish("esp32_htj/humidity", humString);

    Serial.println("published...");
  }  
}

- ESP32 보드가 켜지자마자 데이터 전송이 시작되었다.

    <img src="images/image_013.png"><br>

- 이제 데이터베이스(InfluxDB)에 저장되게 할 수 있다.

    <img src="images/image_014.png"><br>

- 실제로 DB에 데이터가 잘 쌓이고 있는 것을 확인할 수 있다.

    <img src="images/image_015.png"><br>

- 이렇게 쌓이고 있는 데이터를 가시화시켜서 web page로 보여줄 수도 있다.
    - Django 프레임워크를 이용해서 python으로 web application을 프로그램 할 수 있다.

    <img src="images/image_016.png"><br>